In [ ]:
# Add project src to path.
import set_path

# Import modules.
import numpy as np

from src.utils.labels import Labels
from src.utils.las_utils import read_las, label_and_save_las, get_bbox_from_las_file, get_tilecode_from_filename
from src.scrapers.ams_bgt_scraper import scrape_amsterdam_bgt, parse_polygons
from src.fusion.car_fuser import CarFuser

In [ ]:
# The already labelled ground and building points must be removed.
exclude_labels = (Labels.GROUND, Labels.BUILDING)

In [ ]:
# Select the file to process.
in_file = '../datasets/pointcloud/grown_2397_9705.las'
out_file = '../datasets/pointcloud/cars_2397_9705.las'

# Data folder for the fusers.
ahn_data_folder = '../datasets/ahn/'

# Read the point cloud
tilecode = get_tilecode_from_filename(in_file)
pointcloud = read_las(in_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T

In [ ]:
# Specify the bounding box of the area we want to scrape.
bbox = get_bbox_from_las_file(in_file)

# Road parking spots layers in BGT
bgt_layers = ["BGT_WGL_rijbaan_lokale_weg", "BGT_WGL_parkeervlak",
             "BGT_WGL_rijbaan_autoweg", "BGT_WGL_rijbaan_autosnelweg",
             "BGT_WGL_rijbaan_regionale_weg"]

# Scrape data from the Amsterdam WFS and parse the json.
bgt_road_polygons = []
for layer in bgt_layers:
    # Scrape data from the Amsterdam WFS, this will return a json response.
    json_content = scrape_amsterdam_bgt(layer, bbox=bbox)
    
    # Parse the downloaded json response.
    parsed_content, _ = parse_polygons(json_content, offset_meter=-0.3)
    bgt_road_polygons += parsed_content

In [ ]:
car_fuser = CarFuser(Labels.CAR, exclude_labels, ahn_data_folder, 
                     bgt_road_polygons=bgt_road_polygons, 
                     octree_level=10, min_component_size=100,
                     max_above_ground=3, min_width_thresh=1.5,
                     max_width_thresh=2.55, min_length_thresh=2.0,
                     max_length_thresh=7.0)

In [ ]:
las_labels = car_fuser.get_label_mask(tilecode, points, pointcloud.label)

In [ ]:
label_and_save_las(pointcloud, las_labels, out_file)